## Load Necessary Packages

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 1. Import Dataset Online_Retail


In [104]:
df = pd.read_csv("C:\\Users\\Rog\\Downloads\\Filtered_Online_Retail.csv")    # Import data Online_Retail

In [105]:
df.shape # Kiểm tra kích thước của Dataframe


(168275, 8)

In [106]:
df.head() # Hiển thị (preview) data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,542776,21671,RED SPOT CERAMIC DRAWER KNOB,48,2/1/2011 8:23,1.25,15240.0,United Kingdom
1,542776,21668,RED STRIPE CERAMIC DRAWER KNOB,12,2/1/2011 8:23,1.25,15240.0,United Kingdom
2,542776,21670,BLUE SPOT CERAMIC DRAWER KNOB,12,2/1/2011 8:23,1.25,15240.0,United Kingdom
3,542776,17013D,ORIGAMI ROSE INCENSE IN TUBE,96,2/1/2011 8:23,0.42,15240.0,United Kingdom
4,542776,17096,ASSORTED LAQUERED INCENSE HOLDERS,48,2/1/2011 8:23,0.21,15240.0,United Kingdom


In [107]:
df = df.loc[df['Quantity'] > 0] # lọc dữ liệu chỉ dữ lại hàng có Quantity > 0

# 2. Ready data for analysis

### Handle any NaNs in CustomerID

In [108]:
df['CustomerID'].describe() # thống kê mô tả về CustomerID

count    125256.000000
mean      15262.181860
std        1704.322542
min       12347.000000
25%       13869.000000
50%       15078.000000
75%       16775.000000
max       18287.000000
Name: CustomerID, dtype: float64

In [109]:
df.loc[df['CustomerID'].isna()].head() # Lọc ra những giá trị NaNs trong customerID

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
126,542783,21690,NaN,4,2/1/2011 10:02,0.00,NaN,United Kingdom
127,542784,84795C,NaN,3,2/1/2011 10:04,0.00,NaN,United Kingdom
302,542794,10135,COLOURING PENCILS BROWN TUBE,1,2/1/2011 10:56,2.46,NaN,United Kingdom
303,542794,20662,FIRST CLASS HOLIDAY PURSE,1,2/1/2011 10:56,2.46,NaN,United Kingdom
304,542794,20685,DOORMAT RED RETROSPOT,1,2/1/2011 10:56,14.13,NaN,United Kingdom


In [110]:
df = df.dropna(subset=['CustomerID']) # Loại bỏ giá trị NaNs trong CustomerID

In [111]:
df.shape # Kiểm tra lại Data sau khi đã lọc giá trị NaNs

(125256, 8)

In [112]:
df.head() # Preview Data after removing NaNs

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,542776,21671,RED SPOT CERAMIC DRAWER KNOB,48,2/1/2011 8:23,1.25,15240.0,United Kingdom
1,542776,21668,RED STRIPE CERAMIC DRAWER KNOB,12,2/1/2011 8:23,1.25,15240.0,United Kingdom
2,542776,21670,BLUE SPOT CERAMIC DRAWER KNOB,12,2/1/2011 8:23,1.25,15240.0,United Kingdom
3,542776,17013D,ORIGAMI ROSE INCENSE IN TUBE,96,2/1/2011 8:23,0.42,15240.0,United Kingdom
4,542776,17096,ASSORTED LAQUERED INCENSE HOLDERS,48,2/1/2011 8:23,0.21,15240.0,United Kingdom


### - Customer-Item Matrix

In [132]:
# Tạo ma trận khách hàng - sản phẩm 
customer_item_matrix = df.pivot_table(
    index='CustomerID', # Mỗi hàng sẽ là một khách hàng
    columns='StockCode', # Mỗi cột sẽ là một mã sản phẩm
    values='Quantity', # Giá trị của bảng sẽ là số lượng sản phẩm Quantity
    aggfunc='sum' # Nếu khách hàng mua cùng một sản phẩm thì cộng dồn lại
)

In [133]:
customer_item_matrix.loc[12481:].head() # Lọc các dòng từ customerID từ 12347 trở lên

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214N,90214P,90214R,90214V,90214Y,BANK CHARGES,C2,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0
12483.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
12484.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
12490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
12493.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0


In [134]:
customer_item_matrix.shape # Kiểm tra kích thước ma trận

(2551, 3059)

In [118]:
df['StockCode'].nunique() # Đếm số lượng stockcode trong df

3059

In [135]:
df['CustomerID'].nunique() # Đếm số lượng CustomerID

2551

In [136]:
# Tính tần suất xuất hiện của CustomerID 12348 
customer_item_matrix.loc[12348.0].sum() 
print(float(customer_item_matrix.loc[12348.0].sum()))

269.0


In [137]:
# Chuyển đổi dữ liệu:  
# Nếu khách hàng đã mua (Quantity > 0), đặt thành 1  
# Nếu chưa mua (Quantity = 0), đặt thành 0 
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0) 


C:\Users\Rog\AppData\Local\Temp\ipykernel_14292\3969855250.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)


In [138]:
customer_item_matrix.loc[12481:].head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214N,90214P,90214R,90214V,90214Y,BANK CHARGES,C2,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12481.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12483.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12484.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12490.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12493.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


# 3. Collaborative Filtering

In [139]:
from sklearn.metrics.pairwise import cosine_similarity

## 3.1 User-based Filtering

#### - User-to-User Similarity Matrix

In [140]:
# Tính toán ma trận độ tương đồng giữa khách hàng (User-User Similarity Matrix)
user_user_sim_matrix = pd.DataFrame(
    cosine_similarity(customer_item_matrix) # Sử dụng cosine_similarity
)

In [141]:
user_user_sim_matrix.head() # Preview ma trận

,0,1,2,3,4,5,6,7,8,9,...,2541,2542,2543,2544,2545,2546,2547,2548,2549,2550
0,1.000000,0.071611,0.000000,0.031404,0.0,0.021026,0.133235,0.069886,0.143799,0.023870,...,0.0,0.095019,0.041345,0.0,0.000000,0.0,0.101274,0.0,0.014265,0.030817
1,0.071611,1.000000,0.108465,0.087706,0.0,0.000000,0.248069,0.000000,0.000000,0.066667,...,0.0,0.000000,0.115470,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.172133
2,0.000000,0.108465,1.000000,0.047565,0.0,0.000000,0.000000,0.000000,0.000000,0.108465,...,0.0,0.000000,0.000000,0.0,0.035377,0.0,0.000000,0.0,0.064820,0.000000
3,0.031404,0.087706,0.047565,1.000000,0.0,0.077254,0.054393,0.042796,0.105670,0.029235,...,0.0,0.023275,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.052414,0.075485
4,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.138675,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000


In [142]:
# Thiết lập ma trận
user_user_sim_matrix.columns = customer_item_matrix.index # Gán tên cột là CustomerID

user_user_sim_matrix['CustomerID'] = customer_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('CustomerID')

In [143]:
user_user_sim_matrix.head() # Hiển thị 5 dòng ma trận user-user

CustomerID,12347.0,12348.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,12359.0,12360.0,...,18257.0,18260.0,18263.0,18270.0,18272.0,18273.0,18280.0,18281.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12347.0,1.000000,0.071611,0.000000,0.031404,0.0,0.021026,0.133235,0.069886,0.143799,0.023870,...,0.0,0.095019,0.041345,0.0,0.000000,0.0,0.101274,0.0,0.014265,0.030817
12348.0,0.071611,1.000000,0.108465,0.087706,0.0,0.000000,0.248069,0.000000,0.000000,0.066667,...,0.0,0.000000,0.115470,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.172133
12350.0,0.000000,0.108465,1.000000,0.047565,0.0,0.000000,0.000000,0.000000,0.000000,0.108465,...,0.0,0.000000,0.000000,0.0,0.035377,0.0,0.000000,0.0,0.064820,0.000000
12352.0,0.031404,0.087706,0.047565,1.000000,0.0,0.077254,0.054393,0.042796,0.105670,0.029235,...,0.0,0.023275,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.052414,0.075485
12353.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.138675,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000


#### - Xây dựng hệ thống gợi ý cho User-based

In [147]:
# Độ tương tự của khách hàng 15240 với khách hàng khác 
user_user_sim_matrix.loc[15240.0].sort_values(ascending=False)

CustomerID
15240.0    1.000000
16407.0    0.246885
15462.0    0.244949
16434.0    0.243432
13748.0    0.230940
             ...   
18218.0    0.000000
18211.0    0.000000
18210.0    0.000000
12353.0    0.000000
12407.0    0.000000
Name: 15240.0, Length: 2551, dtype: float64

In [ ]:
# Giả dụ ta xét khách hàng A với CustomerID 15240
# Danh sách các sản phẩm mà khách hàng A đã mua 
items_bought_by_A = set(customer_item_matrix.loc[15240.0]
                        [customer_item_matrix.loc[15240.0] > 0].index)

items_bought_by_A

{'16161P',
 '17013D',
 '17021',
 '17096',
 '20914',
 '21216',
 '21218',
 '21485',
 '21668',
 '21669',
 '21670',
 '21671',
 '21672',
 '21843',
 '21877',
 '22077',
 '22083',
 '22084',
 '22423',
 '22457',
 '22469',
 '22835',
 '22961',
 '23084',
 '23100',
 '23108',
 '23112',
 '23203',
 '79321',
 '85123A'}

In [150]:
# Giả xử ta xét một khách hàng B với CustomerID 17404
# Danh sách các sản phẩm mà khách hàng B đã mua
items_bought_by_B = set(customer_item_matrix.loc[17404.0]
                        [customer_item_matrix.loc[17404.0] > 0].index)
items_bought_by_B

{'15036',
 '20974',
 '20975',
 '21212',
 '21213',
 '21704',
 '21791',
 '21889',
 '21977',
 '22139',
 '22358',
 '22416',
 '22417',
 '22489',
 '22491',
 '22492',
 '22530',
 '22536',
 '22551',
 '22554',
 '22651',
 '22720',
 '22721',
 '22752',
 '22915',
 '22938',
 '22949',
 '22951',
 '22961',
 '23080',
 '23081',
 '23154',
 '23159',
 '23176',
 '23177',
 '23191',
 '23193',
 '23194',
 '84077',
 '84987',
 '84988',
 '84991',
 '85178'}

In [151]:
# Đề xuất sản phẩm cho khách hàng A 
# Tìm các sản phẩm mà khách hàng B đã mua nhưng khách hàng A chưa mua
items_to_recommend_to_A = items_bought_by_B - items_bought_by_A

In [152]:
items_to_recommend_to_A # Đề xuất sản phẩm cho khách hàng A

{'15036',
 '20974',
 '20975',
 '21212',
 '21213',
 '21704',
 '21791',
 '21889',
 '21977',
 '22139',
 '22358',
 '22416',
 '22417',
 '22489',
 '22491',
 '22492',
 '22530',
 '22536',
 '22551',
 '22554',
 '22651',
 '22720',
 '22721',
 '22752',
 '22915',
 '22938',
 '22949',
 '22951',
 '23080',
 '23081',
 '23154',
 '23159',
 '23176',
 '23177',
 '23191',
 '23193',
 '23194',
 '84077',
 '84987',
 '84988',
 '84991',
 '85178'}

In [ ]:
# Tạo danh sách sản phẩm đề xuất cho khách hàng A
df.loc[ 
    df['StockCode'].isin(items_to_recommend_to_A), # lọc ra StockCode
    ['StockCode', 'Description'] # Lấy 2 cột StockCode và Description
].drop_duplicates().set_index('StockCode') # Bỏ trùng lặp và đặt StockCode làm đề mục

,Description
StockCode,
22417,PACK OF 60 SPACEBOY CAKE CASES
22139,RETROSPOT TEA SET CERAMIC 11 PC
21977,PACK OF 60 PINK PAISLEY CAKE CASES
21212,PACK OF 72 RETROSPOT CAKE CASES
22951,60 CAKE CASES DOLLY GIRL DESIGN
22358,KINGS CHOICE TEA CADDY
84988,SET OF 72 PINK HEART PAPER DOILIES
22720,SET OF 3 CAKE TINS PANTRY DESIGN
84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS


## 3.2 Item-based Filtering

#### - Ma trận tương đồng sản phẩm - sản phẩm

In [ ]:
# Tạo ma trận độ tương đồng giữa sản phẩm dựa trên dữ liệu mua hàng của khách hàng
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T))

In [158]:
item_item_sim_matrix.columns = customer_item_matrix.T.index
# Gán tên cột là mã sản phẩm (StockCode)
item_item_sim_matrix['StockCode'] = customer_item_matrix.T.index
# Thêm một cột chứa mã sản phẩm (StockCode) để dùng làm chỉ mục
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')
# Thiết lập StockCode làm chỉ mục chính của ma trận

In [163]:
item_item_sim_matrix # Xem ma trận

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214N,90214P,90214R,90214V,90214Y,BANK CHARGES,C2,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.0,0.101015,0.0,0.0,0.00000,0.174964,0.040757,0.000000,0.138013,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031497,0.0,0.000000
10080,0.000000,1.0,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.133631,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10120,0.101015,0.0,1.000000,0.0,0.0,0.00000,0.164957,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10123C,0.000000,0.0,0.000000,1.0,0.0,0.00000,0.308607,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10124A,0.000000,0.0,0.000000,0.0,1.0,0.57735,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BANK CHARGES,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.133631,0.000000,...,0.707107,0.707107,0.707107,0.707107,0.707107,1.000000,0.000000,0.083333,0.0,0.000000
C2,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.050508,0.069007,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.062994,0.0,0.020261
M,0.031497,0.0,0.000000,0.0,0.0,0.00000,0.051434,0.035944,0.044544,0.152145,...,0.117851,0.117851,0.117851,0.117851,0.117851,0.083333,0.062994,1.000000,0.0,0.053606


#### - Xây dựng hệ thống gợi ý cho Item-based

In [201]:
# Tạo ma trận khách hàng - sản phẩm
customer_item_matrix = df.pivot_table(index="CustomerID", columns="StockCode", values="Quantity", fill_value=0)

# Tính toán độ tương tự giữa sản phẩm
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T), 
                                    index=customer_item_matrix.columns, 
                                    columns=customer_item_matrix.columns)

item_item_sim_matrix

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214N,90214P,90214R,90214V,90214Y,BANK CHARGES,C2,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.0,0.004667,0.0,0.0,0.000000,0.701142,0.246778,0.000000,0.022515,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000204,0.0,0.000000
10080,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.013975,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10120,0.004667,0.0,1.000000,0.0,0.0,0.000000,0.004081,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10123C,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.025584,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10124A,0.000000,0.0,0.000000,0.0,1.0,0.784465,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BANK CHARGES,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.011539,0.000000,...,0.707107,0.707107,0.707107,0.707107,0.707107,1.000000,0.000000,0.009979,0.0,0.000000
C2,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.067847,0.054099,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.006846,0.0,0.010573
M,0.000204,0.0,0.000000,0.0,0.0,0.000000,0.025446,0.003256,0.004178,0.048793,...,0.014113,0.014113,0.014113,0.014113,0.014113,0.009979,0.006846,1.000000,0.0,0.008357


#### Đưa ra gợi ý

In [202]:
# Giả sử khách hàng mua sản phẩm với StockCode 22708
# Tìm ra 10 sản phẩm gần giống nhất với sản phẩm có StockCode 22708
top_10_similar_items = list(
    item_item_sim_matrix.loc["22708"] 
    .sort_values(ascending=False)
    .iloc[:10]
    .index
)

In [ ]:
top_10_similar_items # Đưa ra 10 sản phẩm giống 22708

['22708',
 '22308',
 '85071C',
 '21902',
 '23176',
 '22981',
 '21899',
 '21900',
 '22615',
 '23242']

In [205]:
# Đưa ra các thông tin mô tả sản phẩm
df.loc[
    df['StockCode'].isin(top_10_similar_items), 
    ['StockCode', 'Description']
].drop_duplicates().set_index('StockCode').loc[top_10_similar_items]

,Description
StockCode,
22708,WRAP DOLLY GIRL
22308,TEA COSY BLUE STRIPE
85071C,"CHARLIE+LOLA""EXTREMELY BUSY"" SIGN"
21902,"KEY FOB , FRONT DOOR"
23176,ABC TREASURE BOOK BOX
22981,PANTRY APPLE CORER
21899,"KEY FOB , GARAGE DESIGN"
21900,"KEY FOB , SHED"
22615,PACK OF 12 CIRCUS PARADE TISSUES
